In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [3]:
spx_path = "data/SPX_database_2010.xlsx"
df_tri = pd.read_excel(spx_path, sheet_name="total_ret").set_index("Dates")
df_tri.index = pd.to_datetime(df_tri.index)

In [7]:
tickers = [
    "MSFT", "AAPL", "NVDA", "GOOGL", "IBM",
    "AVGO", "ADBE", "CRM", "INTC", "INTU",
    "UNH", "PFE", "LLY", "ABT", "MRK",
    "MCD", "AMZN", "COST", "SBUX", "PG",
    "JPM", "GS", "MS", "BLK", "CAT",
    "DE", "GE", "XOM", "NEE", "ETN"
]

def clean_column_names(df):
    new_columns = []
    for col in df.columns:
        if ' UN Equity' in str(col):
            ticker = col.split(' UN Equity')[0]
            new_columns.append(ticker)
        elif ' UW Equity' in str(col):
            ticker = col.split(' UW Equity')[0]
            new_columns.append(ticker)
        else:
            new_columns.append(col)
    return new_columns

df_tri.columns = clean_column_names(df_tri)

# tickers = [t for t in tickers if t in df_tri.columns]
print(tickers)

['MSFT', 'AAPL', 'NVDA', 'GOOGL', 'IBM', 'AVGO', 'ADBE', 'CRM', 'INTC', 'INTU', 'UNH', 'PFE', 'LLY', 'ABT', 'MRK', 'MCD', 'AMZN', 'COST', 'SBUX', 'PG', 'JPM', 'GS', 'MS', 'BLK', 'CAT', 'DE', 'GE', 'XOM', 'NEE', 'ETN']


In [23]:
prices = df_tri[tickers].sort_index()
daily_returns = prices.pct_change().dropna()
daily_returns = daily_returns.loc[:, ~daily_returns.columns.duplicated()]
daily_returns.head()

,MSFT,AAPL,NVDA,GOOGL,IBM,AVGO,ADBE,CRM,INTC,INTU,...,JPM,GS,MS,BLK,CAT,DE,GE,XOM,NEE,ETN
Dates,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.015087,0.015749,-0.010278,0.010893,0.011841,0.031165,0.008425,0.014233,0.023531,0.003256,...,0.029517,0.025114,0.044258,0.027479,0.027374,0.036421,0.021150,0.014077,0.008145,0.011001
2010-01-05,0.000323,0.000928,0.014712,-0.004399,-0.012080,0.007423,0.016441,-0.004277,-0.000479,0.000000,...,0.019370,0.017677,0.036557,0.004316,0.011955,-0.003746,0.005178,0.003905,-0.009768,-0.005908
2010-01-06,-0.006138,-0.015908,0.006397,-0.025232,-0.006496,0.007895,-0.002107,-0.001745,-0.003355,-0.013957,...,0.005495,-0.010671,0.012796,-0.020617,0.003038,-0.002686,-0.005151,0.008644,0.009865,-0.004379
2010-01-07,-0.009424,-0.001845,-0.019492,-0.023257,-0.003462,-0.006266,-0.019403,-0.004437,-0.009616,-0.003292,...,0.019809,0.019567,0.014484,0.010994,0.004037,0.007182,0.051780,-0.003143,-0.002817,0.014766
2010-01-08,0.005904,0.006648,0.002161,0.013318,0.010035,0.007357,-0.005438,0.001486,0.011164,0.026420,...,-0.002456,-0.018911,-0.020352,0.007040,0.011228,0.027273,0.021539,-0.004011,-0.008665,0.022447


In [9]:
def annualize_return(mean_daily_return, trading_days=252):
    return (1 + mean_daily_return) ** trading_days - 1

def annualize_cov(daily_cov, trading_days=252):
    return daily_cov * trading_days

In [ ]:
from scipy.optimize import minimize

def solve_dynamic_markowitz(expected_returns, cov_matrix, target_vol,
                            max_weight_asset=0.25, max_weight_cash=0.50):
    """
    expected_returns: (n,) annualized expected returns for risky assets
    cov_matrix: (n, n) annualized covariance of risky assets
    returns weights: (n+1,) for [assets..., cash]
    """
    n_assets = len(expected_returns)
    
    # extend covariance to include cash
    extended_cov = np.zeros((n_assets + 1, n_assets + 1))
    extended_cov[:n_assets, :n_assets] = cov_matrix
    extended_mu = np.append(expected_returns, 0.0)  # cash return = 0
    
    # bounds
    bounds = [(0.0, max_weight_asset)] * n_assets + [(0.0, max_weight_cash)]
    
    # constraint: sum w = 1
    def cons_weights(w):
        return np.sum(w) - 1.0
    
    constraints = [{
        "type": "eq",
        "fun": cons_weights
    }]
    
    # volatility constraint
    def cons_vol(w):
        port_var = np.dot(w, np.dot(extended_cov, w))
        port_vol = np.sqrt(port_var)
        return target_vol - port_vol
    
    constraints.append({
        "type": "ineq",
        "fun": cons_vol
    })
    
    # objective: maximize return -> minimize negative
    def objective(w):
        return -np.dot(w, extended_mu)
    
    # initial guess: spread across assets, rest in cash
    x0 = np.array([min(1.0 / (n_assets + 1), max_weight_asset)] * n_assets + [0.0])
    x0[-1] = 1.0 - np.sum(x0[:-1])
    x0[-1] = min(x0[-1], max_weight_cash)
    if np.sum(x0) < 1.0:
        gap = 1.0 - np.sum(x0)
        x0[:-1] += gap / n_assets
    
    res = minimize(
        objective,
        x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 500, "ftol": 1e-9, "disp": False}
    )
    
    # retry with slightly looser vol if needed
    if not res.success:
        relaxed_target = target_vol + 0.005
        def cons_vol_relaxed(w):
            port_var = np.dot(w, np.dot(extended_cov, w))
            port_vol = np.sqrt(port_var)
            return relaxed_target - port_vol
        constraints[-1] = {"type": "ineq", "fun": cons_vol_relaxed}
        res = minimize(
            objective,
            x0,
            method="SLSQP",
            bounds=bounds,
            constraints=constraints,
            options={"maxiter": 500, "ftol": 1e-9, "disp": False}
        )
    
    return res.x, res.success

In [14]:
lookback_days = 125
rebal_freq = "M"
target_annual_vol = 0.08
max_weight_asset = 0.25
max_weight_cash = 0.50

# month-end rebal dates
month_ends = daily_returns.resample(rebal_freq).last().index

valid_rebal_dates = []
for d in month_ends:
    window = daily_returns.loc[:d].tail(lookback_days)
    if len(window) == lookback_days:
        valid_rebal_dates.append(d)
valid_rebal_dates = pd.DatetimeIndex(valid_rebal_dates)

valid_rebal_dates

/var/folders/pm/ps47mfm943x_nqjshcjsk5ch0000gn/T/ipykernel_60867/2562391438.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_ends = daily_returns.resample(rebal_freq).last().index


DatetimeIndex(['2010-06-30', '2010-07-31', '2010-08-31', '2010-09-30',
               '2010-10-31', '2010-11-30', '2010-12-31', '2011-01-31',
               '2011-02-28', '2011-03-31',
               ...
               '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28',
               '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30',
               '2025-07-31', '2025-08-31'],
              dtype='datetime64[ns]', length=183, freq=None)

In [ ]:
weights_history = {}
portfolio_returns = []

all_dates = daily_returns.index

for i, reb_date in enumerate(valid_rebal_dates):
    # 1) lookback window
    window = daily_returns.loc[:reb_date].tail(lookback_days)
    
    # 2) drop assets without full data in the window
    window = window.dropna(axis=1)
    
    # 3) enforce unique columns for this window
    #    (sometimes .dropna keeps dups if original DF had them)
    cols_this_reb = pd.Index(window.columns).unique()
    window = window[cols_this_reb]
    
    # stop if too few assets
    if window.shape[1] < 2:
        continue
    
    # 4) expected returns
    mean_daily = window.mean()
    exp_ret_annual = annualize_return(mean_daily)
    
    # 5) covariance
    daily_cov = window.cov()
    annual_cov = annualize_cov(daily_cov)
    
    # 6) optimization
    w_opt, ok = solve_dynamic_markowitz(
        exp_ret_annual.values,
        annual_cov.values,
        target_annual_vol,
        max_weight_asset=max_weight_asset,
        max_weight_cash=max_weight_cash
    )
    
    n_assets_this_reb = len(cols_this_reb)
    assert len(w_opt) == n_assets_this_reb + 1, "optimizer returned unexpected size"
    
    asset_names = list(cols_this_reb) + ["CASH"]
    weights_history[reb_date] = pd.Series(w_opt, index=asset_names)
    
    if i < len(valid_rebal_dates) - 1:
        next_reb_date = valid_rebal_dates[i + 1]
        period_dates = all_dates[(all_dates > reb_date) & (all_dates <= next_reb_date)]
    else:
        period_dates = all_dates[all_dates > reb_date]
    
    period_rets = daily_returns.loc[period_dates, cols_this_reb]
    period_rets = period_rets.fillna(0.0)
    
    assert period_rets.shape[1] == n_assets_this_reb, (
        f"period_rets has {period_rets.shape[1]} cols, "
        f"but weights expect {n_assets_this_reb}"
    )
    
    # 9) portfolio return
    port_rets = period_rets.dot(w_opt[:-1])   # exclude cash
    port_rets.name = "portfolio"
    
    portfolio_returns.append(port_rets)

# 10) concat all periods
portfolio_returns = pd.concat(portfolio_returns).sort_index()
portfolio_returns.head()


Dates
2010-07-01    0.001623
2010-07-02   -0.003035
2010-07-05    0.000000
2010-07-06    0.000149
2010-07-07    0.012787
Name: portfolio, dtype: float64

In [25]:
def compute_performance(series, rf=0.0):
    # series is daily returns
    ann_ret = (1 + series).prod() ** (252.0 / len(series)) - 1
    ann_vol = series.std() * np.sqrt(252)
    if ann_vol > 0:
        sharpe = (ann_ret - rf) / ann_vol
    else:
        sharpe = np.nan
    return ann_ret, ann_vol, sharpe

ann_ret, ann_vol, sharpe = compute_performance(portfolio_returns)

print("Dynamic Markowitz on 30 stocks")
print(f"Annualized return: {ann_ret:.2%}")
print(f"Annualized vol   : {ann_vol:.2%}")
print(f"Sharpe (rf=0)    : {sharpe:.2f}")


Dynamic Markowitz on 30 stocks
Annualized return: 13.74%
Annualized vol   : 10.55%
Sharpe (rf=0)    : 1.30
